# Forecasting Ensembles & Composition in sktime

Ensembles combine multiple forecasters to improve robustness and accuracy. Composition
lets you chain transformations and models into a single workflow.


## Ensemble intuition
Given forecasts $\hat{y}^{(1)}, \hat{y}^{(2)}, \dots$, a weighted ensemble is:

\[\hat{y}^{ens} = \sum_{m=1}^M w_m \hat{y}^{(m)}, \quad \sum_m w_m = 1\]

Weights can be uniform, error-based, or learned (stacking).


## Simulate two baselines and an ensemble


In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

rng = np.random.default_rng(7)

n = 120
season = 12
series = pd.Series(
    30 + 6 * np.sin(2 * np.pi * np.arange(n) / season) + rng.normal(0, 1.2, n),
    index=pd.period_range("2012-01", periods=n, freq="M"),
)

# Hold out last 24 points
train = series.iloc[:-24]
test = series.iloc[-24:]

# Naive and seasonal naive baselines
naive_forecast = pd.Series([train.iloc[-1]] * len(test), index=test.index)
seasonal_template = train.iloc[-season:].values
seasonal_naive = pd.Series(
    [seasonal_template[i % season] for i in range(len(test))],
    index=test.index,
)

ensemble = 0.5 * naive_forecast + 0.5 * seasonal_naive


In [2]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index.to_timestamp(), y=train, mode="lines", name="Train"))
fig.add_trace(go.Scatter(x=test.index.to_timestamp(), y=test, mode="lines", name="Test"))
fig.add_trace(go.Scatter(x=naive_forecast.index.to_timestamp(), y=naive_forecast, mode="lines", name="Naive"))
fig.add_trace(go.Scatter(x=seasonal_naive.index.to_timestamp(), y=seasonal_naive, mode="lines", name="Seasonal Naive"))
fig.add_trace(go.Scatter(x=ensemble.index.to_timestamp(), y=ensemble, mode="lines", name="Ensemble"))
fig.update_layout(title="Ensemble vs Baselines", xaxis_title="Time", yaxis_title="y")
fig.show()


In [3]:
def mae(y_true, y_pred):
    return float(np.mean(np.abs(y_true - y_pred)))

print("MAE - Naive:", mae(test, naive_forecast))
print("MAE - Seasonal Naive:", mae(test, seasonal_naive))
print("MAE - Ensemble:", mae(test, ensemble))

# Weight sweep to show how ensembles can be tuned
weights = np.linspace(0, 1, 41)
mae_curve = [mae(test, w * naive_forecast + (1 - w) * seasonal_naive) for w in weights]
best_idx = int(np.argmin(mae_curve))

fig = go.Figure()
fig.add_trace(go.Scatter(x=weights, y=mae_curve, mode="lines+markers", name="MAE"))
fig.add_trace(
    go.Scatter(
        x=[weights[best_idx]],
        y=[mae_curve[best_idx]],
        mode="markers",
        name="Best weight",
    )
)
fig.update_layout(
    title="Ensemble Weight Sweep (Naive vs Seasonal Naive)",
    xaxis_title="Weight on Naive",
    yaxis_title="MAE",
)
fig.show()


MAE - Naive: 3.934244175340366
MAE - Seasonal Naive: 1.2139413746873176
MAE - Ensemble: 1.9922014053496209


## Composition patterns in sktime
- **TransformedTargetForecaster**: apply transformations (detrend, deseasonalize, box-cox)
  before fitting a forecaster.
- **ForecastingPipeline**: chain exogenous feature transformations + forecaster.
- **MultiplexForecaster**: select among candidate models at fit time.
- **EnsembleForecaster / StackingForecaster**: average or stack multiple models.


## Optional: sktime ensemble API


In [4]:
try:
    from sktime.forecasting.compose import EnsembleForecaster, MultiplexForecaster, TransformedTargetForecaster
    from sktime.forecasting.naive import NaiveForecaster
    from sktime.forecasting.theta import ThetaForecaster
    from sktime.transformations.series.detrend import Detrender
    from sktime.forecasting.trend import PolynomialTrendForecaster

    forecasters = [
        ("naive", NaiveForecaster(strategy="last")),
        ("theta", ThetaForecaster()),
    ]

    ensemble = EnsembleForecaster(forecasters=forecasters)
    multiplex = MultiplexForecaster(forecasters=forecasters)

    pipeline = TransformedTargetForecaster(
        steps=[("detrend", Detrender(forecaster=PolynomialTrendForecaster(degree=1))), ("model", ThetaForecaster())]
    )

    # Fit/predict pattern
    # ensemble.fit(train)
    # y_pred = ensemble.predict(fh)
except Exception as e:
    print("sktime ensemble demo skipped:", e)


sktime ensemble demo skipped: No module named 'sktime'
